**M ARISH**

**SC25M159**

Optimization Techniques Algorithms

**5. Rank-1 Update (DFP)**

In [1]:
import numpy as np

def sr1_quadratic_robust(Q, b, x0, tol=1e-8, max_iter=1000, enable_history=True):
    """
    Robust SR1 for f(x) = 0.5 * (x^T Q x - x^T b), g(x) = Qx - 0.5 b.
    Safeguards: Armijo backtracking, descent check, curvature test on SR1 update.
    """
    x = x0.astype(np.float64)
    n = len(x)
    H = np.eye(n, dtype=np.float64)
    g = Q @ x - 0.5 * b
    history = [] if enable_history else None

    def f(xv):
        return 0.5 * (xv @ Q @ xv - xv @ b)

    # Armijo backtracking line search
    def line_search(xv, pv, gv, alpha_init=1.0, c=1e-4, rho=0.5, alpha_min=1e-12, alpha_max=1.0):
        alpha = min(alpha_init, alpha_max)
        fx = f(xv)
        descent = gv @ pv
        if descent >= 0:  # not a descent direction; fall back to steepest descent
            pv = -gv
            descent = gv @ pv
        # Backtrack until sufficient decrease or alpha too small
        while f(xv + alpha * pv) > fx + c * alpha * descent:
            alpha *= rho
            if alpha < alpha_min:
                return None, pv  # signal failure
        return alpha, pv

    for k in range(max_iter):
        gnorm = np.linalg.norm(g)
        if gnorm < tol:
            return x, k, f(x), history

        # Search direction from SR1 inverse Hessian
        p = -H @ g

        # Backtracking line search (robust)
        alpha, p = line_search(x, p, g, alpha_init=1.0)
        if alpha is None:
            # Fallback: steepest descent step with conservative alpha
            p = -g
            alpha, _ = line_search(x, p, g, alpha_init=1.0)
            if alpha is None:
                # If even SD fails, stop to avoid NaN cascades
                break

        # Step
        x_new = x + alpha * p
        g_new = Q @ x_new - 0.5 * b

        s = x_new - x
        y = g_new - g

        # SR1 update with curvature safeguard
        Hy = H @ y
        u = s - Hy
        denom = u @ y
        if abs(denom) > 1e-12:
            H = H + np.outer(u, u) / denom
            # Optional: clip extreme H entries to avoid overflow
            # H = np.clip(H, -1e12, 1e12)

        # Update state
        x, g = x_new, g_new
        if enable_history:
            history.append(x.copy())

        # Optional: reset H if it becomes ill-conditioned
        if not np.isfinite(H).all():
            H = np.eye(n, dtype=np.float64)

    return x, k + 1, f(x), history


# Problem data
Q = np.array([[3, 0, 1],
              [0, 4, 2],
              [1, 2, 3]], dtype=np.float64)
b = np.array([3, 0, 1], dtype=np.float64)

# Start with any x0
x0 = np.array([0.0, 0.0, 0.0], dtype=np.float64)

x_sr1, iters, f_sr1, hist = sr1_quadratic_robust(Q, b, x0, tol=1e-10, max_iter=1000, enable_history=True)
print("SR1 Solution:", x_sr1)
print("Iterations:", iters)
print("Final f(x):", f_sr1)

x_exact = np.linalg.solve(Q, 0.5 * b)
print("Exact solution:", x_exact)
print("Error norm:", np.linalg.norm(x_sr1 - x_exact))

SR1 Solution: [ 5.00000000e-01  8.67361738e-17 -1.29236899e-16]
Iterations: 4
Final f(x): -0.375
Exact solution: [0.5 0.  0. ]
Error norm: 1.9118392072088212e-16
